In [12]:
import threading
import pandas as pd
import pandas_datareader.data as web
import datetime
from sklearn import linear_model
from time import sleep
import joblib

In [13]:
# this class is to give colors to the print statements
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


# This function will give the price after the delay of sleep_time units of seconds
def getprice(label="WIPRO.NS", sleep_time=1):
    # Pause for sleep_time number of seconds
    sleep(sleep_time)
    # Using yahoo finance to get the stock market live data
    quote = web.get_quote_yahoo(label)
    price = quote["price"].values[0]
    current_time = datetime.datetime.now()
    return tuple([price, current_time])


# This function trains the model using the input data(dataframe)
def train(input):
    print("\nModel updating...", end=" ")
    # We take last column of the features as target and rest are taken as attributes
    featureMat = input.iloc[:, : len(input.columns) - 1]
    label = input[input.columns[-1]]
    # Here we are using linear regression model
    model = linear_model.LinearRegression()
    model.fit(featureMat, label)
    # Model is being written on the hard drive
    joblib.dump(model, "modelLR.pkl")
    print("[Completed]")

# Training over  
Increase the values of these variables to get improved results
but if we will increase them then we'll have to wait for (number_of_features X training_record_criteran) X steep_time units seconds for first training


In [14]:
number_of_features = 5  # This indicates how many columns the dataframe will have.
training_record_criterian = 5  # This decides how frequently the model will update [5 new features -> retrain the model]
number_of_predictions = 3  # Tells how many prediction in series you want

In [ ]:
data = pd.DataFrame(columns=range(number_of_features))  # creating an empty dataframe

predict_input = list()

while 1:

    feature = list()  # stores the features for a single record for dataframe

    for i in range(number_of_features):

        price = getprice()[0]
        feature.append(price)
        predict_input.append(price)

        try:  # this will throw exception in two cases:
            # 1> model is not yet trained and saved
            # 2> model prediction is not working.

            first_predict = True  # flag for detecting the first prediction in predicted series
            model = joblib.load("modelLR.pkl")  # trying to open the saved model (can throw exception)
            print("")
            inputlist = predict_input.copy()  # copying the list to make the prediction if model is ready
            #   printing latest 3 prices
            for feature_value in inputlist[-(3):]:
                print(f"{bcolors.WARNING} --> ", int(feature_value * 100) / 100, end=" ")
            #   taking the latest price
            price = getprice(sleep_time=0)[0]
            #   Starting the predictions
            for i in range(number_of_predictions):
                pre_price = model.predict([inputlist[-(number_of_features - 1):]])
                #   printing the predicted values one by one in the series
                print(f"{bcolors.OKBLUE} --> ", int(pre_price[0] * 100) / 100, end=" ")
                #   This block will only run for the first prediction in series
                if first_predict:
                    # When prediction tells about increase in price
                    if pre_price[0] - inputlist[-1] > 0:
                        print(f"{bcolors.OKGREEN}  \u2191", end="")
                        #   Calculating the % of increase the program predicts and printing.
                        print(f"{bcolors.BOLD}[", int((pre_price[0] - price) * 1000000 / price) / 10000, "%] ", end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # When prediction says that no change will happen
                    elif pre_price[0] - inputlist[-1] == 0:
                        print(f"{bcolors.HEADER} \u2022", end="")
                        print(f"{bcolors.BOLD}[", int((pre_price[0] - price) * 1000000 / price) / 10000, "%] ", end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # When prediction is about decrease in price
                    else:
                        print(f"{bcolors.FAIL}  \u2193", end="")
                        print(f"{bcolors.BOLD}[", int(-(pre_price[0] - price) * 1000000 / price) / 10000, "%] ",
                              end=" ")
                        print(f"{bcolors.OKCYAN} Actual: ", price, end="")
                    # Next statement talk about what happened actually
                    if price - inputlist[-1] > 0:
                        print(f"{bcolors.OKGREEN}  \u2191", end=" ")
                    elif price - inputlist[-1] == 0:
                        print(f"{bcolors.HEADER} \u2022", end="")
                    else:
                        print(f"{bcolors.FAIL}  \u2193", end=" ")

                    first_predict = False


                #   pushing the predicted price in the back of the input array..
                #   it will be used in predicting next element in the series
                inputlist.append(pre_price[0])
        except:

            print("Please Wait while the model is getting ready...")
    #   Adding the feature in the dataframe
    data.loc[len(data.index)] = feature
    #   If number of elements present in the dataframe is multiple of training record criterian, the retraining
    if len(data.index) % training_record_criterian == 0:
        # print(data)
        #   training in separate thread
        trainer = threading.Thread(target=train, args=(data,))
        trainer.start()
        trainer.join()


 -->  676.1 Please Wait while the model is getting ready...

 -->  676.1  -->  676.1 Please Wait while the model is getting ready...

 -->  676.1  -->  676.1  -->  676.25 Please Wait while the model is getting ready...

 -->  676.1  -->  676.25  -->  675.95  -->  674.86   ↓[ 0.1597 %]   Actual:  675.95 • -->  672.26  -->  665.52 
 -->  676.25  -->  675.95  -->  675.95  -->  676.29   ↑[ 0.0441 %]   Actual:  676.0  ↑  -->  676.74  -->  677.42 
 -->  675.95  -->  675.95  -->  676.0  -->  675.63   ↓[ 0.0766 %]   Actual:  676.15  ↑  -->  674.32  -->  670.89 
 -->  675.95  -->  676.0  -->  676.15  -->  676.26   ↑[ 0.0242 %]   Actual:  676.1  ↓  -->  676.27  -->  676.25 
 -->  676.0  -->  676.15  -->  676.1  -->  675.66   ↓[ 0.064 %]   Actual:  676.1 • -->  674.54  -->  671.66 
 -->  676.15  -->  676.1  -->  676.1  -->  676.16   ↑[ 0.0024 %]   Actual:  676.15  ↑  -->  676.16  -->  675.98 
 -->  676.1  -->  676.1  -->  676.15  -->  676.1   ↓[ 0.0063 %]   Actual:  676.15 • -->  675.81  -->  67

 -->  676.1  -->  676.1  -->  676.2  -->  676.15   ↓[ 0.0059 %]   Actual:  676.2 • -->  676.11  -->  676.16 
 -->  676.1  -->  676.2  -->  676.2  -->  676.14   ↓[ 0.0085 %]   Actual:  676.2 • -->  676.18  -->  676.18 
 -->  676.2  -->  676.2  -->  676.2  -->  676.22   ↑[ -0.0254 %]   Actual:  676.4  ↑  -->  676.2  -->  676.18 
 -->  676.2  -->  676.2  -->  676.4  -->  676.33   ↓[ -0.0058 %]   Actual:  676.3  ↓  -->  676.25  -->  676.37 
 -->  676.2  -->  676.4  -->  676.3  -->  676.22   ↓[ 0.0107 %]   Actual:  676.3 • -->  676.36  -->  676.3 
 -->  676.4  -->  676.3  -->  676.3  -->  676.41   ↑[ 0.0025 %]   Actual:  676.4  ↑  -->  676.33  -->  676.29 
 -->  676.3  -->  676.3  -->  676.4  -->  676.32   ↓[ -0.0038 %]   Actual:  676.3  ↓  -->  676.29  -->  676.36 
 -->  676.3  -->  676.4  -->  676.3  -->  676.27   ↓[ 0.0484 %]   Actual:  676.6  ↑  -->  676.35  -->  676.29 
 -->  676.4  -->  676.3  -->  676.6  -->  676.6   ↑[ 0.0082 %]   Actual:  676.55  ↓  -->  676.42  -->  676.59 
 -->  

 -->  677.45  -->  677.5  -->  677.75  -->  677.73   ↓[ 0.0388 %]   Actual:  678.0  ↑  -->  677.72  -->  677.87 
 -->  677.5  -->  677.75  -->  678.0  -->  677.91   ↓[ -0.0241 %]   Actual:  677.75  ↓  -->  678.0  -->  678.15 
 -->  677.75  -->  678.0  -->  677.8  -->  677.92   ↑[ 0.0187 %]   Actual:  677.8 • -->  678.09  -->  677.97 
 -->  678.0  -->  677.8  -->  677.7  -->  677.92   ↑[ -0.0029 %]   Actual:  677.95  ↑  -->  677.85  -->  677.81 
 -->  677.8  -->  677.7  -->  677.9  -->  677.83   ↓[ 0.0095 %]   Actual:  677.9 • -->  677.79  -->  677.95 
 -->  677.7  -->  677.9  -->  677.9  -->  677.84   ↓[ 0.008 %]   Actual:  677.9 • -->  677.98  -->  678.0 
 -->  677.9  -->  677.9  -->  677.8  -->  677.95   ↑[ 0.0224 %]   Actual:  677.8 • -->  677.98  -->  677.93 
 -->  677.9  -->  677.8  -->  677.8  -->  677.87   ↑[ -0.0113 %]   Actual:  677.95  ↑  -->  677.85  -->  677.88 
 -->  677.8  -->  677.8  -->  677.95  -->  677.91   ↓[ -0.0459 %]   Actual:  677.6  ↓  -->  677.92  -->  678.03 


 -->  679.1  -->  679.25  -->  679.1  -->  679.24   ↑[ 0.0217 %]   Actual:  679.1 • -->  679.35  -->  679.33 
 -->  679.25  -->  679.1  -->  679.1  -->  679.26   ↑[ -0.0197 %]   Actual:  679.4  ↑  -->  679.25  -->  679.32 
 -->  679.1  -->  679.1  -->  679.4  -->  679.33   ↓[ 0.0091 %]   Actual:  679.4 • -->  679.39  -->  679.56 
 -->  679.1  -->  679.4  -->  679.4  -->  679.43   ↑[ 0.0346 %]   Actual:  679.2  ↓  -->  679.6  -->  679.63 
 -->  679.4  -->  679.4  -->  679.2  -->  679.45   ↑[ 0.0085 %]   Actual:  679.4  ↑  -->  679.5  -->  679.47 
 -->  679.4  -->  679.2  -->  679.4  -->  679.46   ↑[ 0.0168 %]   Actual:  679.35  ↓  -->  679.44  -->  679.59 
 -->  679.2  -->  679.4  -->  679.35  -->  679.37   ↑[ 0.0035 %]   Actual:  679.35 • -->  679.52  -->  679.55 
 -->  679.4  -->  679.35  -->  679.35  -->  679.51   ↑[ -0.0419 %]   Actual:  679.8  ↑  -->  679.53  -->  679.59 
 -->  679.35  -->  679.35  -->  679.8  -->  679.71   ↓[ 0.0124 %]   Actual:  679.8 • -->  679.76  -->  679.99 
